$ \huge{1.}$ We take 10 indicator decision variables named c0, c1, c2,...,c9, each denoting one of the courses C0, C1, C2,...,C9 respectively.  
A variable ci takes the value 1 if the corresponding course Ci is chosen, else that variable is assigned the value 0.  
Different courses are assigned different credits. These credits are denoted by the variables w0, w1, w2,...,w9, where wi denoted the credits associated with the course Ci.  
We are provided some statements using which we frame our constraints:  
  
A minimum of 26 credits need to be gathered over the semester, which means:  
w0c0 + w1c1 + w2c2 + w3c3 + w4c4 + w5c5 + w6c6 + w7c7 + w8c8 + w9c9 >= 26  
  
At least 4 courses and a maximum of 8 courses are to be taken. So:  
c0 + c1 + c2 + c3 + c4 + c5 + c6 + c7 + c8 + c9 >= 4  
and  
c0 + c1 + c2 + c3 + c4 + c5 + c6 + c7 + c8 + c9 <= 8  
  
C2 and C7 run during the same slot, so both of them cannot be taken simultaneously, which means only one of these courses can be opted. This means:  
c2 + c7 <= 1, becuse if one of them is taken, the other cannot be taken, or both of them may not be taken.  
  
Similar case exists for courses C1 and C9. Thus, by the same logic, we obtain:  
c1 + c9 <= 1  
  
At least one and at most 2 courses are to be taken out of C0, C1, C2, C3. So we get the following constraints:  
c0 + c1 + c2 + c3 >= 1  and  
c0 + c1 + c2 + c3 <= 2  
  
We can register for at most one course among the courses C5 and C9. This means:  
c5 + c9 <= 1  
  
We can register for at most 2 courses out of C6, C8, and C9. So:  
c6 + c8 + c9 <= 2  
  
If one registers for C2, one cannot register for C8. Therefore:  
c2 + c8 <= 1  
  
If we register for C9, we must also register for C4, which means:  
c9 - c4 <= 0

$ \huge{2.}$ Model construction:

In [1]:
!pip install -q pyomo

     |████████████████████████████████| 9.7 MB 5.3 MB/s 
     |████████████████████████████████| 49 kB 6.2 MB/s 


In [2]:
from pyomo.environ import *

In [3]:
import numpy as np

In [4]:
model_lab3_Ex2 = ConcreteModel()

In [5]:
M=11
N=10

In [6]:
obj_coef_w = np.array([4,3,6,5,2,6,4,8,8,6])

In [7]:
constr_coef_A = np.array([[-4,-3,-6,-5,-2,-6,-4,-8,-8,-6],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[1,1,1,1,1,1,1,1,1,1],[0,0,1,0,0,0,0,1,0,0],[0,1,0,0,0,0,0,0,0,1],[-1,-1,-1,-1,0,0,0,0,0,0],[1,1,1,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,1],[0,0,0,0,0,0,1,0,1,1],[0,0,1,0,0,0,0,0,1,0],[0,0,0,0,-1,0,0,0,0,1]])

In [8]:
constr_rhs_b = np.array([-26,-4,8,1,1,-1,2,1,2,1,0])

In [9]:
#lower_bound = np.array([])
#upper_bound = np.array([]) #These are not needed as all the variables are Binary and can only take values of either 0 or 1

In [10]:
col_indices = np.arange(N)

In [11]:
row_indices = np.arange(M)

In [12]:
model_lab3_Ex2.c = Var(col_indices, domain=Binary)

In [13]:
model_lab3_Ex2.constraints = ConstraintList()

In [14]:
for i in row_indices:
  model_lab3_Ex2.constraints.add(sum(constr_coef_A[i][j]*model_lab3_Ex2.c[j] for j in col_indices) <= constr_rhs_b[i])

In [15]:
#for j in col_indices:
#  model_lab3_Ex2.c[j].setlb(lower_bound[j])
#  model_lab3_Ex2.c[j].setub(upper_bound[j]) #Not needed for Binary variables

In [16]:
model_lab3_Ex2.objective = Objective(expr = sum(obj_coef_w[j]*model_lab3_Ex2.c[j] for j in col_indices), sense=maximize)

In [17]:
model_lab3_Ex2.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    c : Size=10, Index=c_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True : Binary
          1 :     0 :  None :     1 : False :  True : Binary
          2 :     0 :  None :     1 : False :  True : Binary
          3 :     0 :  None :     1 : False :  True : Binary
          4 :     0 :  None :     1 : False :  True : Binary
          5 :     0 :  None :     1 : False :  True : Binary
          6 :     0 :  None :     1 : False :  True : Binary
          7 :     0 :  None :     1 : False :  True : Binary
          8 :     0 :  None :   

In [18]:
!apt-get install -y -qq coinor-cbc

Selecting previously unselected package coinor-libcoinutils3v5.
(Reading database ... 155676 files and directories currently installed.)
Preparing to unpack .../0-coinor-libcoinutils3v5_2.10.14+repack1-1_amd64.deb ...
Unpacking coinor-libcoinutils3v5 (2.10.14+repack1-1) ...
Selecting previously unselected package coinor-libosi1v5.
Preparing to unpack .../1-coinor-libosi1v5_0.107.9+repack1-1_amd64.deb ...
Unpacking coinor-libosi1v5 (0.107.9+repack1-1) ...
Selecting previously unselected package coinor-libclp1.
Preparing to unpack .../2-coinor-libclp1_1.16.11+repack1-1_amd64.deb ...
Unpacking coinor-libclp1 (1.16.11+repack1-1) ...
Selecting previously unselected package coinor-libcgl1.
Preparing to unpack .../3-coinor-libcgl1_0.59.10+repack1-1_amd64.deb ...
Unpacking coinor-libcgl1 (0.59.10+repack1-1) ...
Selecting previously unselected package coinor-libcbc3.
Preparing to unpack .../4-coinor-libcbc3_2.9.9+repack1-1_amd64.deb ...
Unpacking coinor-libcbc3 (2.9.9+repack1-1) ...
Selecting p

In [19]:
opt_cbc = SolverFactory('cbc')

In [20]:
result = opt_cbc.solve(model_lab3_Ex2)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)



Problem: 
- Name: unknown
  Lower bound: 37.0
  Upper bound: 37.0
  Number of objectives: 1
  Number of constraints: 11
  Number of variables: 10
  Number of binary variables: 10
  Number of integer variables: 10
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.05174994468688965
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.05174994468688965


In [21]:
# display solution
print('\nObjective = ', model_lab3_Ex2.objective())

print('\nDecision Variables')
for i in col_indices:
  print('c[',i,'] = ', model_lab3_Ex2.c[i].value)

print('\nConstraints')
model_lab3_Ex2.constraints.display()


Objective =  37.0

Decision Variables
c[ 0 ] =  1.0
c[ 1 ] =  0.0
c[ 2 ] =  0.0
c[ 3 ] =  1.0
c[ 4 ] =  1.0
c[ 5 ] =  1.0
c[ 6 ] =  1.0
c[ 7 ] =  1.0
c[ 8 ] =  1.0
c[ 9 ] =  0.0

Constraints
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -37.0 : -26.0
      2 :  None :  -7.0 :  -4.0
      3 :  None :   7.0 :   8.0
      4 :  None :   1.0 :   1.0
      5 :  None :   0.0 :   1.0
      6 :  None :  -2.0 :  -1.0
      7 :  None :   2.0 :   2.0
      8 :  None :   1.0 :   1.0
      9 :  None :   2.0 :   2.0
     10 :  None :   1.0 :   1.0
     11 :  None :  -1.0 :   0.0


$ \huge{3.}$ The optimal values of the variables and the
objective function value are as follows:  
  
c[0] =  1.0  
c[1] =  0.0  
c[2] =  0.0  
c[3] =  1.0  
c[4] =  1.0  
c[5] =  1.0  
c[6] =  1.0  
c[7] =  1.0  
c[8] =  1.0  
c[9] =  0.0  
  
Objective =  37.0

$ \huge{4.}$ Modified problem

In [22]:
model_lab3_Ex2_4 = ConcreteModel()

In [23]:
M=11
N=10

In [24]:
obj_coef_w = np.array([4,3,6,5,2,6,4,8,8,6])

In [25]:
constr_coef_A = np.array([[-4,-3,-6,-5,-2,-6,-4,-8,-8,-6],[-1,-1,-1,-1,-1,-1,-1,-1,-1,-1],[1,1,1,1,1,1,1,1,1,1],[0,0,1,0,0,0,0,1,0,0],[0,1,0,0,0,0,0,0,0,1],[-1,-1,-1,-1,0,0,0,0,0,0],[1,1,1,1,0,0,0,0,0,0],[0,0,0,0,0,1,0,0,0,1],[0,0,0,0,0,0,1,0,1,1],[0,0,1,0,0,0,0,0,1,0],[0,0,0,0,-1,0,0,0,0,1]])

In [26]:
constr_rhs_b = np.array([-26,-4,8,1,1,-1,2,1,2,1,0])

In [27]:
lower_bound = np.array([0,0,0,0,0,0,0,0,0,0])
upper_bound = np.array([1,1,1,1,1,1,1,1,1,1]) 

In [28]:
col_indices = np.arange(N)

In [29]:
row_indices = np.arange(M)

In [30]:
model_lab3_Ex2_4.c = Var(col_indices)

In [31]:
model_lab3_Ex2_4.constraints = ConstraintList()

In [32]:
for i in row_indices:
  model_lab3_Ex2_4.constraints.add(sum(constr_coef_A[i][j]*model_lab3_Ex2_4.c[j] for j in col_indices) <= constr_rhs_b[i])

In [33]:
for j in col_indices:
  model_lab3_Ex2_4.c[j].setlb(lower_bound[j])
  model_lab3_Ex2_4.c[j].setub(upper_bound[j]) #Setting bounds for real valued decision variables

In [34]:
model_lab3_Ex2_4.objective = Objective(expr = sum(obj_coef_w[j]*model_lab3_Ex2_4.c[j] for j in col_indices), sense=maximize)

In [35]:
model_lab3_Ex2_4.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   11 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11}

1 Var Declarations
    c : Size=10, Index=c_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :  None :     1 : False :  True :  Reals
          1 :     0 :  None :     1 : False :  True :  Reals
          2 :     0 :  None :     1 : False :  True :  Reals
          3 :     0 :  None :     1 : False :  True :  Reals
          4 :     0 :  None :     1 : False :  True :  Reals
          5 :     0 :  None :     1 : False :  True :  Reals
          6 :     0 :  None :     1 : False :  True :  Reals
          7 :     0 :  None :     1 : False :  True :  Reals
          8 :     0 :  None :   

In [36]:
opt_cbc = SolverFactory('cbc')

In [37]:
result = opt_cbc.solve(model_lab3_Ex2_4)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)
print('Solver time:',result.solver.time)



Problem: 
- Name: unknown
  Lower bound: 37.0
  Upper bound: 37.0
  Number of objectives: 1
  Number of constraints: 12
  Number of variables: 11
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 4
  Error rc: 0
  Time: 0.04130697250366211
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal
Solver time: 0.04130697250366211


In [38]:
# display solution
print('\nObjective = ', model_lab3_Ex2_4.objective())

print('\nDecision Variables')
for i in col_indices:
  print('c[',i,'] = ', model_lab3_Ex2_4.c[i].value)

print('\nConstraints')
model_lab3_Ex2_4.constraints.display()


Objective =  37.0

Decision Variables
c[ 0 ] =  1.0
c[ 1 ] =  0.0
c[ 2 ] =  0.0
c[ 3 ] =  1.0
c[ 4 ] =  1.0
c[ 5 ] =  1.0
c[ 6 ] =  1.0
c[ 7 ] =  1.0
c[ 8 ] =  1.0
c[ 9 ] =  0.0

Constraints
constraints : Size=11
    Key : Lower : Body  : Upper
      1 :  None : -37.0 : -26.0
      2 :  None :  -7.0 :  -4.0
      3 :  None :   7.0 :   8.0
      4 :  None :   1.0 :   1.0
      5 :  None :   0.0 :   1.0
      6 :  None :  -2.0 :  -1.0
      7 :  None :   2.0 :   2.0
      8 :  None :   1.0 :   1.0
      9 :  None :   2.0 :   2.0
     10 :  None :   1.0 :   1.0
     11 :  None :  -1.0 :   0.0


$ \huge{5.}$  The solution to MILP may not always be obtained by rounding off the solution obtained by LP.  
This is so because rounding off the solution of LP might give us an infeasible solution as the solution tuple obtained on rounding off might lie outside of our feasible region.  
Further, even if we do get a feasible solution on rounding off the solution obtained by LP, it might not be the optimal solution to our MILP problem.

$ \huge{6.}$ By introducing a new course, we must introduce this into our objective function. Also, we will have to add constraints regarding this new course.  
The modified model is solved below:

In [39]:
model_lab3_Ex2.c10 = Var(domain=Binary) #declaring new variable

In [40]:
model_lab3_Ex2.constraints.add(expr= model_lab3_Ex2.c10 + model_lab3_Ex2.c[5] <= 1)
model_lab3_Ex2.constraints.add(expr= model_lab3_Ex2.c10 + model_lab3_Ex2.c[6] <= 1)
model_lab3_Ex2.constraints.add(expr= 12*model_lab3_Ex2.c10 + summation(obj_coef_w,model_lab3_Ex2.c) >=26) #Adding new constraints to ConstraintList

In [41]:
model_lab3_Ex2.objective6 = Objective(expr= 12*model_lab3_Ex2.c10 + summation(obj_coef_w,model_lab3_Ex2.c), sense=maximize) #Modifying the objective

In [42]:
model_lab3_Ex2.constraints[1].deactivate() #Deactivating redundant constraints

In [43]:
model_lab3_Ex2.objective.deactivate() #Deactivating the old objective

In [44]:
model_lab3_Ex2.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   14 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}

2 Var Declarations
    c : Size=10, Index=c_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False :  True : Binary
          1 :     0 :   0.0 :     1 : False :  True : Binary
          2 :     0 :   0.0 :     1 : False :  True : Binary
          3 :     0 :   1.0 :     1 : False :  True : Binary
          4 :     0 :   1.0 :     1 : False :  True : Binary
          5 :     0 :   1.0 :     1 : False :  True : Binary
          6 :     0 :   1.0 :     1 : False :  True : Binary
          7 :     0 :   1.0 :     1 : False :  True : Binary
          8 :     0 

In [45]:
result = opt_cbc.solve(model_lab3_Ex2)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 13
  Number of variables: 11
  Number of binary variables: 11
  Number of integer variables: 11
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
    Black box: 
      Number of iterations: 0
  Error rc: 0
  Time: 0.024214506149291992
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [46]:
# display solution
print('\nObjective = ', model_lab3_Ex2.objective6())

print('\nDecision Variables')
for i in col_indices:
  print('c[',i,'] = ', model_lab3_Ex2.c[i].value)
print('c[ 10 ] = ', model_lab3_Ex2.c10.value)

print('\nConstraints')
model_lab3_Ex2.constraints.display()


Objective =  45.0

Decision Variables
c[ 0 ] =  1.0
c[ 1 ] =  0.0
c[ 2 ] =  0.0
c[ 3 ] =  1.0
c[ 4 ] =  1.0
c[ 5 ] =  0.0
c[ 6 ] =  0.0
c[ 7 ] =  1.0
c[ 8 ] =  1.0
c[ 9 ] =  1.0
c[ 10 ] =  1.0

Constraints
constraints : Size=14
    Key : Lower : Body : Upper
      2 :  None : -6.0 :  -4.0
      3 :  None :  6.0 :   8.0
      4 :  None :  1.0 :   1.0
      5 :  None :  1.0 :   1.0
      6 :  None : -2.0 :  -1.0
      7 :  None :  2.0 :   2.0
      8 :  None :  1.0 :   1.0
      9 :  None :  2.0 :   2.0
     10 :  None :  1.0 :   1.0
     11 :  None :  0.0 :   0.0
     12 :  None :  1.0 :   1.0
     13 :  None :  1.0 :   1.0
     14 :  26.0 : 45.0 :  None


$ \huge{7.}$ The optimal values of the variables and objective
function value are reported below:  
  
c[0] =  1.0  
c[1] =  0.0  
c[2] =  0.0  
c[3] =  1.0  
c[4] =  1.0  
c[5] =  0.0  
c[6] =  0.0  
c[7] =  1.0  
c[8] =  1.0  
c[9] =  1.0  
c[10] =  1.0    

    
Objective =  45.0

$ \huge{8.}$  Removing the integer constraint from the above model, we get the new model as shown below:

In [47]:
model_lab3_Ex2_4.c10 = Var() #declaring new variable

In [48]:
model_lab3_Ex2_4.c10.setlb(0)
model_lab3_Ex2_4.c10.setub(1)

In [49]:
model_lab3_Ex2_4.constraints.add(expr= model_lab3_Ex2_4.c10 + model_lab3_Ex2_4.c[5] <= 1)
model_lab3_Ex2_4.constraints.add(expr= model_lab3_Ex2_4.c10 + model_lab3_Ex2_4.c[6] <= 1)
model_lab3_Ex2_4.constraints.add(expr= 12*model_lab3_Ex2_4.c10 + summation(obj_coef_w,model_lab3_Ex2_4.c) >=26) #Adding new constraints to ConstraintList

In [50]:
model_lab3_Ex2_4.objective8 = Objective(expr= 12*model_lab3_Ex2_4.c10 + summation(obj_coef_w,model_lab3_Ex2_4.c), sense=maximize) #Modifying the objective

In [51]:
model_lab3_Ex2_4.constraints[1].deactivate() #Deactivating redundant constraints

In [52]:
model_lab3_Ex2_4.objective.deactivate() #Deactivating the old objective

In [53]:
model_lab3_Ex2_4.pprint()

2 Set Declarations
    c_index : Size=1, Index=None, Ordered=False
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   10 : {0, 1, 2, 3, 4, 5, 6, 7, 8, 9}
    constraints_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :   14 : {1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14}

2 Var Declarations
    c : Size=10, Index=c_index
        Key : Lower : Value : Upper : Fixed : Stale : Domain
          0 :     0 :   1.0 :     1 : False :  True :  Reals
          1 :     0 :   0.0 :     1 : False :  True :  Reals
          2 :     0 :   0.0 :     1 : False :  True :  Reals
          3 :     0 :   1.0 :     1 : False :  True :  Reals
          4 :     0 :   1.0 :     1 : False :  True :  Reals
          5 :     0 :   1.0 :     1 : False :  True :  Reals
          6 :     0 :   1.0 :     1 : False :  True :  Reals
          7 :     0 :   1.0 :     1 : False :  True :  Reals
          8 :     0 

In [54]:
result = opt_cbc.solve(model_lab3_Ex2_4)
print(result)

print('Solver status:', result.solver.status)
print('Solver termination condition:',result.solver.termination_condition)


Problem: 
- Name: unknown
  Lower bound: 45.0
  Upper bound: 45.0
  Number of objectives: 1
  Number of constraints: 14
  Number of variables: 12
  Number of nonzeros: 11
  Sense: maximize
Solver: 
- Status: ok
  User time: -1.0
  System time: 0.0
  Wallclock time: 0.0
  Termination condition: optimal
  Termination message: Model was solved to optimality (subject to tolerances), and an optimal solution is available.
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: None
      Number of created subproblems: None
    Black box: 
      Number of iterations: 4
  Error rc: 0
  Time: 0.0372767448425293
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Solver status: ok
Solver termination condition: optimal


In [55]:
# display solution
print('\nObjective = ', model_lab3_Ex2_4.objective8())

print('\nDecision Variables')
for i in col_indices:
  print('c[',i,'] = ', model_lab3_Ex2_4.c[i].value)
print('c[ 10 ] = ', model_lab3_Ex2_4.c10.value)

print('\nConstraints')
model_lab3_Ex2_4.constraints.display()


Objective =  45.0

Decision Variables
c[ 0 ] =  1.0
c[ 1 ] =  0.0
c[ 2 ] =  0.0
c[ 3 ] =  1.0
c[ 4 ] =  1.0
c[ 5 ] =  0.0
c[ 6 ] =  0.0
c[ 7 ] =  1.0
c[ 8 ] =  1.0
c[ 9 ] =  1.0
c[ 10 ] =  1.0

Constraints
constraints : Size=14
    Key : Lower : Body : Upper
      2 :  None : -6.0 :  -4.0
      3 :  None :  6.0 :   8.0
      4 :  None :  1.0 :   1.0
      5 :  None :  1.0 :   1.0
      6 :  None : -2.0 :  -1.0
      7 :  None :  2.0 :   2.0
      8 :  None :  1.0 :   1.0
      9 :  None :  2.0 :   2.0
     10 :  None :  1.0 :   1.0
     11 :  None :  0.0 :   0.0
     12 :  None :  1.0 :   1.0
     13 :  None :  1.0 :   1.0
     14 :  26.0 : 45.0 :  None


In this case, we can see that the solution to the MILP can be obtained by rounding off the solution obtained by LP as the bounds placed on the decision variables in the LP are respected even after rounding off and they hold good for the MILP. Also, the values of the decision variables of the LP in this case are exactly similar to the ones obtained in the corresponding MILP, so roudning off will not really have an effect on the optimal solution.  
This however, is not always true as sometimes rounding off the LP solutions might result in the decision variables taking variables that lie outside of their predefined bounds and/or constraints.